In [5]:
import random
from enum import StrEnum
from typing import Self
from rich import print as rp 

class Names(StrEnum):
    JOHN = "John Lennon"
    PAUL = "Paul McCartney"
    GEORGE = "George Harrison"
    RINGO = "Ringo Starr"

class Beetle:
    health_points: int
    max_health_points: int
    name: Names
    damage_min: int
    damage_max: int

    def __init__(
        self,
        health_points: int = 100,
        max_health_points: int = 100,
        name: Names = Names.JOHN,
        damage_min: int = 5,
        damage_max: int = 15,
    ) -> None:
        self.health_points = health_points
        self.max_health_points = max_health_points
        self.name = name
        self.damage_min = damage_min
        self.damage_max = damage_max

    def __eq__(self, other: Self) -> bool:
        return self.health_points == other.health_points

    def __lt__(self, other: Self) -> bool:
        return self.health_points < other.health_points

    def __le__(self, other: Self) -> bool:
        return self.health_points <= other.health_points

    def __str__(self) -> str:
        return f'Beetle(name="{self.name}", hp={self.health_points}/{self.max_health_points})'

    def is_alive(self) -> bool:
        return self.health_points > 0

    def styling(self) -> str:
        if self.name == Names.JOHN:
            return "in Johny style"
        elif self.name is Names.PAUL:
            return "in McCartney style"
        return "without style"

    def attack(self, other: Self) -> bool:
        damage = random.randint(self.damage_min, self.damage_max)
        
        rp(
            f"[bold yellow]{self.name}[/bold yellow] атакует [bold red]{other.name}[/bold red] "
            f"{self.styling()} на [bold]{damage}[/bold] урона",
        )
        other.health_points -= damage
        
        if not other.is_alive():
            rp(f"[bold red]{other.name} умер![/bold red]")
            
            old_hp = self.health_points
            self.health_points = min(self.health_points + 10, self.max_health_points)
            bonus = self.health_points - old_hp
            
            if bonus > 0:
                rp(
                    f"[bold green]{self.name} получил +{bonus} HP! "
                    f"Теперь {self.health_points}/{self.max_health_points} HP[/bold green]"
                )
            
            return True
        
        print(
            f"У {other.name} осталось {other.health_points}/{other.max_health_points} HP"
        )
        return False


class BeetlesArmy:
    beetles_list: list[Beetle]
    beetles_name: Names
    beetles_max_health_points: int
    damage_min: int
    damage_max: int

    def __init__(
        self,
        beetles_name: Names,
        beetles_army_size: int = 20,
        beetles_max_health_points: int = 100,
        damage_min: int = 5,
        damage_max: int = 15,
    ):
        self.beetles_list = []
        self.beetles_name = beetles_name
        self.beetles_max_health_points = beetles_max_health_points
        self.damage_min = damage_min
        self.damage_max = damage_max

        for _ in range(beetles_army_size):
            beetle = Beetle(
                health_points=random.randint(1, self.beetles_max_health_points),
                max_health_points=self.beetles_max_health_points,
                name=self.beetles_name,
                damage_min=self.damage_min,
                damage_max=self.damage_max,
            )
            self.beetles_list.append(beetle)

    def __len__(self) -> int:
        return len(self.beetles_list)

    def __add__(self, other: Self) -> Self:
        if self.beetles_name != other.beetles_name:
            raise ValueError("Cannot make two different-named beetles friends")
        new_beetles_list: list[Beetle] = self.beetles_list + other.beetles_list
        new_army = self.__class__(
            beetles_army_size=1,
            beetles_name=self.beetles_name,
            beetles_max_health_points=self.beetles_max_health_points,
        )
        new_army.beetles_list = new_beetles_list
        return new_army

    def __eq__(self, other: Self) -> bool:
        return len(self.get_alive_beetles()) == len(other.get_alive_beetles())

    def __lt__(self, other: Self) -> bool:
        return len(self.get_alive_beetles()) < len(other.get_alive_beetles())

    def __le__(self, other: Self) -> bool:
        return len(self.get_alive_beetles()) <= len(other.get_alive_beetles())

    def get_alive_beetles(self) -> list[Beetle]:
        return [b for b in self.beetles_list if b.is_alive()]

    def total_health(self) -> int:
        return sum(b.health_points for b in self.get_alive_beetles())


    def print_battle_status(self, other: Self, round_num: int):
        alive1 = self.get_alive_beetles()
        alive2 = other.get_alive_beetles()
        
        rp(
            "Статус битвы:",
            f"[bold cyan]Раунд {round_num}[/bold cyan]\n\n",
            f"[yellow]{self.beetles_name}[/yellow]: {len(alive1)} жуков",
            f"([green]{self.total_health()} HP[/green])\n",
            f"[yellow]{other.beetles_name}[/yellow]: [bold]{len(alive2)}[/bold] жуков",
            f"([green]{other.total_health()} HP[/green])"
        )

    def battle(self, other: Self):
        rp("\n")
        rp("[bold red]БИТВА НАЧИНАЕТСЯ![/bold red]")
        rp("\n")
        
        round_num = 1
        
        while True:
            alive_army1 = self.get_alive_beetles()
            alive_army2 = other.get_alive_beetles()
            
            if not alive_army1:
                rp("\n")
                rp("[bold green]БИТВА ОКОНЧЕНА![/bold green]")
                rp(
                    f"Армия [bold yellow]{other.beetles_name}[/bold yellow] одержала победу!\n\n"
                    f"Выживших жуков: [bold green]{len(alive_army2)}[/bold green]\n"
                    f"Суммарное HP: [bold green]{other.total_health()}[/bold green]",
                )
                return other
            
            if not alive_army2:
                rp("\n")
                rp("[bold green]БИТВА ОКОНЧЕНА![/bold green]")
                rp(
                    f"Армия [bold yellow]{self.beetles_name}[/bold yellow] одержала победу!\n\n"
                    f"Выживших жуков: [bold green]{len(alive_army1)}[/bold green]\n"
                    f"Суммарное HP: [bold green]{self.total_health()}[/bold green]",
                )
                return self
            
            self.print_battle_status(other, round_num)
            
            attacker = random.choice(alive_army1)
            defender = random.choice(alive_army2)
            rp(f"\n[bold blue]------> Ход армии {self.beetles_name}[/bold blue]<------")
            attacker.attack(defender)
            
            alive_army2 = other.get_alive_beetles()
            if not alive_army2:
                continue
            
            alive_army1 = self.get_alive_beetles()
            if alive_army1:
                attacker = random.choice(alive_army2)
                defender = random.choice(alive_army1)
                rp(f"\n[bold blue]------> Ход армии {other.beetles_name}[/bold blue]<------")
                attacker.attack(defender)
            
            round_num += 1
            print()


def get_user_input():
    rp(
            "[bold cyan]Добро пожаловать в симулятор битвы жуков![/bold cyan]\n\n"
            "Создайте две армии",
    )
    
    rp("\n[bold]Доступные имена жуков:[/bold]")
    for i, name in enumerate(Names, 1):
        rp(f"  {i}. {name.value}")
    
    armies = []
    
    for i in range(2):
        rp(f"\n[bold yellow]--- Создание армии {i + 1} ---[/bold yellow]")
        
        name_choice = int(input("Выберите имя (1-4): "))
        name = list(Names)[name_choice - 1]
        
        army_size = int(input("Размер армии: "))
        
        max_hp = int(input("Максимальное здоровье у жуков: "))
        
        damage_min = int(input("Минимальный урон: "))
        
        damage_max = int(input("Максимальный урон: "))
        
        army = BeetlesArmy(
            beetles_name=name,
            beetles_army_size=army_size,
            beetles_max_health_points=max_hp,
            damage_min=damage_min,
            damage_max=damage_max,
        )
        
        armies.append(army)
        rp(f"[green]Армия {name} создана![/green]")
    
    return armies


if __name__ == "__main__":
    armies = get_user_input()
    
    ba1, ba2 = armies
    
    rp("\n[bold]Сравнение армий:[/bold]")
    if ba1 > ba2:
        rp(f"[green]Армия {ba1.beetles_name} сильнее (больше живых жуков)[/green]")
    elif ba1 < ba2:
        rp(f"[green]Армия {ba2.beetles_name} сильнее (больше живых жуков)[/green]")
    else:
        rp("[yellow]Армии равны по силе[/yellow]")
    
    print("\n")
    ready = input("Начать битву? [да/нет]:")
    
    if ready.lower() == "да":
        winner = ba1.battle(ba2)
    else:
        rp("[yellow]Ссыкло[/yellow]")

Добро пожаловать в симулятор битвы жуков!

Создайте две армии

Доступные имена жуков:

1. John Lennon

2. Paul McCartney

3. George Harrison

...

George Harrison атакует John Lennon without style на 23 урона

John Lennon умер!

George Harrison получил +10 HP! Теперь 33/44 HP

БИТВА ОКОНЧЕНА!

Армия George Harrison одержала победу!

Выживших жуков: 16
Суммарное HP: 415